In [36]:
import pint
from tinarm import NameQuantityPair
from tinarm import Quantity as OldQuantity
from tinarm import Unit as OldUnit
import tinarm
import logging
import time
import requests
import uuid
import random

LOGGING_LEVEL = logging.INFO


### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
logger.info(f"tinarm version {tinarm.__version__}")
q = pint.UnitRegistry()


2024-01-18 12:44:20,281 - NoJobId - INFO - Martins-MacBook-Air.local - 800482112.py-><module>() - tinarm version 0.1


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [37]:
import yaml

with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)

# Stator

In [51]:
stator_parameters = {
    "slot_liner_thikness": 200 *q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 4 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 136 * q.mm,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 1 * q.mm,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

## Rotor
Surface mounted Breadloaf magnets

In [49]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 250 * q.micrometer,
    }

## Simulation Parameters

In [40]:
simulation_parameters = {
       "samples_per_electrical_period": 180 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [41]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [42]:
op_parameters = {
    "simulated_speed": 2060 * q.rpm,
    "current_density": 0 * q.A / q.mm ** 2,
    "current_angle": 0 * q.degrees
    }

In [43]:
class Unit:
    def __init__(self, name: str, exponent: int):
        self.name = name
        self.exponent = int(exponent)

    def to_dict(self):
        return {
            "name": self.name,
            "exponent": self.exponent,
        }
    
class Quantity:
    def __init__(self, magnitude, units: list[Unit]):
        self.magnitude = magnitude
        self.units = [Unit(*u) if type(u) != Unit else u for u in units]

    def to_dict(self):
        return {
            "magnitude": self.magnitude,
            "units": [u.to_dict() for u in self.units],
        }

In [44]:
OldQuantity(4242, [Unit("millimeter", 2), Unit("second", -1)])


In [45]:
class Machine(object):
    def __init__(self, stator, rotor, winding):

        self.stator = stator
        self.rotor = rotor
        self.winding = winding

    def __repr__(self) -> str:
        return f"Machine({self.stator_parameters}, {self.rotor_parameters}, {self.winding_parameters})"
    
    def to_api(self):
        stator_api = [NameQuantityPair("stator",
                                        k,
                                        Quantity(*self.stator[k].to_tuple())) for k in  self.stator]
        rotor_api = [NameQuantityPair("rotor",
                                        k,
                                        Quantity(*self.rotor[k].to_tuple())) for k in  self.rotor]
        winding_api = [NameQuantityPair("winding",
                                        k,
                                        Quantity(*self.winding[k].to_tuple())) for k in  self.winding]
        data = []
        data.extend(list(x.to_dict() for x in stator_api))
        data.extend(list(x.to_dict() for x in rotor_api))
        data.extend(list(x.to_dict() for x in winding_api))
        return data
        
    
class Job(object):
    def __init__(self, machine: Machine, operating_point, simulation, title=None):
        if title is None:
            self.title = self.generate_title()
        else:
            self.title = title
        self.type = "electromagnetic_spmbrl_fscwseg"
        self.status = 0
        self.machine = machine
        self.operating_point = operating_point
        self.simulation = simulation
        
    def __repr__(self) -> str:
        return f"Job({self.machine}, {self.operating_point}, {self.simulation})"
    
    def generate_title(self):
        "gets a random title from the wordlists"
        random_offset = random.randint(500, 286797)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Adjectives-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        adjective = random.choice(response.text.splitlines()[1:-1])
        random_offset = random.randint(500, 871742)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Nouns-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        noun = random.choice(response.text.splitlines()[1:-1])
        title = f"{adjective}-{noun}"
        return title 

    def to_api(self):
        job =  {"status": 0,
                "title": self.title,
                "type": self.type,
                "tasks": 11,
                "data": []}
        
        operating_point_api = [NameQuantityPair("operating_point",
                                        k,
                                        Quantity(*op_parameters[k].to_tuple())) for k in  op_parameters]
        
        simulation_api = [NameQuantityPair("simulation",
                                        k,
                                        Quantity(*simulation_parameters[k].to_tuple())) for k in  simulation_parameters]
        job["data"].extend(list(x.to_dict() for x in operating_point_api))
        job["data"].extend(list(x.to_dict() for x in simulation_api))
        job["data"].extend(self.machine.to_api())
        return job
    
    def run(self):
        pass

In [46]:
class Api:
    """
    The TAE User API
    """

    def __init__(self, root_url, api_key):
        """
        Initialize the API
        """
        self._root_url = root_url
        self._api_key = api_key
        self._org_id = ""#"659bf5a7b7135efed48311ba"

        logger.info(f"root_url: {self._root_url}")

    def get_job(self, job_id):
        """
        Get a job from the TAE API
        """
        response = requests.get(
            url=f"{self._root_url}/jobs/{job_id}?apikey={self._api_key}",
        )
        response.raise_for_status()
        return response.json()
    
    def create_job(self, job):
        """
        Create a job for the TAE API
        """
        response = requests.post(
            url=f"{self._root_url}/jobs/?apikey={self._api_key}&org_id={self._org_id}",
            json=job.to_api()
        )
        response.raise_for_status()
        return response.json()
    

In [47]:
api = Api(config["root_url"], config["api_key"])

2024-01-18 12:44:20,677 - NoJobId - INFO - Martins-MacBook-Air.local - 1293051725.py->__init__() - root_url: http://localhost:4301


In [48]:
m1 = Machine(stator_parameters, rotor_parameters, winding_parameters)

j1 = Job(m1, op_parameters, simulation_parameters)

api.create_job(j1)


{'id': '65a90f15b0696af2d6bf98be',
 'owner_user_id': '65a6502069420b5a7777bf26',
 'creation_date': '2024-01-18T11:44:21Z',
 'status': 0,
 'title': 'costive-craniology',
 'type': 'electromagnetic_spmbrl_fscwseg',
 'data': [{'section': 'operating_point',
   'name': 'simulated_speed',
   'value': {'magnitude': 2060,
    'units': [{'name': 'revolutions_per_minute', 'exponent': 1}]}},
  {'section': 'operating_point',
   'name': 'current_density',
   'value': {'magnitude': 0,
    'units': [{'name': 'ampere', 'exponent': 1},
     {'name': 'millimeter', 'exponent': -2}]}},
  {'section': 'operating_point',
   'name': 'current_angle',
   'value': {'magnitude': 0, 'units': [{'name': 'degree', 'exponent': 1}]}},
  {'section': 'simulation',
   'name': 'samples_per_electrical_period',
   'value': {'magnitude': 180,
    'units': [{'name': 'count', 'exponent': 1},
     {'name': 'turn', 'exponent': -1}]}},
  {'section': 'simulation',
   'name': 'timestep_intervals',
   'value': {'magnitude': 180, 'unit